In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as utils
import getData

In [2]:
senLen = 53
numClasses = None
indexSize = 10662
embeddingSize = 300
kernelSizes = [3,4,5]
numFilters = 3
embeddings = None
XY = None
model = None
lossFn = nn.CrossEntropyLoss().type(torch.FloatTensor)
opt = None

In [3]:
size = [embeddingSize,200,300,500,1000,2]

for i in range(len(conv_layer_sizes) - 1):
            layer = nn.Conv1d(conv_layer_sizes[i], conv_layer_sizes[i + 1], \
                              conv_windows_sizes[i], stride=conv_strides_sizes[i])
            self.encoder.add_module("layer-" + str(i + 1), layer)
 
            bn = nn.BatchNorm1d(conv_layer_sizes[i + 1])
            self.encoder.add_module("bn-" + str(i + 1), bn)
            self.encoder.add_module("activation-" + str(i + 1), activation)
            

SyntaxError: invalid character in identifier (<ipython-input-3-2189af73a447>, line 4)

In [27]:

def makeEmbeddings(numWords,index,X,senWords,embed):
    inputData = torch.zeros([len(X),1,senWords,embeddingSize],dtype=torch.float64)
    for i,sen in enumerate(X):
        senIndex = torch.zeros([1,senWords,embeddingSize],dtype=torch.float64)
        for j,word in enumerate(sen.split()):
            senIndex[0,j]= embed[index[word]]
        inputData[i][0] = Variable(torch.DoubleTensor(senIndex[0]))
    return inputData

def makeModel(lr):
    model = nn.Sequential(
            nn.Conv2d(1,200,kernel_size=(3,embeddingSize),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(200),
            nn.Dropout(p=0.2),
            nn.Conv2d(200,300,(4,1),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(300),
            nn.Dropout(p=0.2),
            nn.Conv2d(300,500,(5,1),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(500),
            nn.Dropout(p=0.2),
            Flatten(),
            
    )
    opt = optim.Adam(model.parameters(),lr=lr)
    return model,opt

def train(numEpochs=1):
    for epoch in range(numEpochs):
        model.train()
        for i,(x,y) in enumerate(XY):
            xVar = Variable(x).float()
            yVar = Variable(y).type(torch.LongTensor)
            scores = model(xVar)
            loss = lossFn(scores,yVar)
            if (i + 1) % 100 == 0:
                print("i = %d, loss = %.4f" % (i + 1, loss.data[0]))
            opt.zero_grad()
            loss.backward()
            opt.step()
        print('starting new epoch')

def checkAccuracy(dataSet):
    #if(self.XY.dataset.train):
    #    print('Checking accuracy on validation set')
    #else:
    #    print('Checking accuracy on test set')
    numCorrect = 0
    numSamples = 0
    model.eval()
    for i,(x,y) in enumerate(dataSet):
        xVar = Variable(x).float()
        yVar = Variable(y).type(torch.LongTensor)
        scores = model(xVar)
        _,preds = scores.data.cpu().max(1)
        numCorrect += (preds == yVar).sum()
        numSamples += preds.size(0)
    acc = float(numCorrect)/numSamples
    print('Got %d / %d correct (%.2f)' % (numCorrect, numSamples, 100 * acc))

class Flatten(nn.Module):
    def forward(self,x):
        N,C,H,W = x.size()
        return x.view(N,-1)

In [5]:
print('calling getData...')
X,index,XY,Y,maxWords = getData.runNew()

calling getData...
13161


In [21]:
print('making embeddings...')
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
embeddings = torch.randn(len(index),300)
c = 0
for k,v in index.items():
    try: 
        embeddings[v].copy_(torch.from_numpy(word_vectors.get_vector(k)))
        c += 1
    except KeyError:
        continue 

making embeddings...


In [28]:
inputData = makeEmbeddings(len(index),index,X,maxWords,embeddings)

In [29]:
inputDataTr = inputData[:8000]
inputDataTe = inputData[8000:len(inputData)]
Ytr = Y[:8000]
Yte = Y[8000:len(Y)]

In [ ]:
print('making model...')
model,opt=makeModel(lr=1e-2)
print(type(inputData), type(Y))
my_datasetTr = utils.TensorDataset(inputDataTr, Ytr)
my_datasetTe = utils.TensorDataset(inputDataTe, Yte)
my_dataloaderTr = utils.DataLoader(my_datasetTr,batch_size=64)
my_dataloaderTe = utils.DataLoader(my_datasetTe,batch_size=64)
XY = my_dataloaderTr

In [37]:
print('calling train...')
train(200)

calling train...


/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


i = 100, loss = 3.4782
starting new epoch
i = 100, loss = 3.5070
starting new epoch
i = 100, loss = 2.9022
starting new epoch
i = 100, loss = 3.4351
starting new epoch
i = 100, loss = 3.3503
starting new epoch
i = 100, loss = 2.7276
starting new epoch
i = 100, loss = 3.6631
starting new epoch
i = 100, loss = 3.5452
starting new epoch
i = 100, loss = 3.6326
starting new epoch
i = 100, loss = 3.7763
starting new epoch
i = 100, loss = 3.2894
starting new epoch
i = 100, loss = 3.1551
starting new epoch
i = 100, loss = 3.5182
starting new epoch
i = 100, loss = 3.3579
starting new epoch
i = 100, loss = 2.9495
starting new epoch
i = 100, loss = 3.7006
starting new epoch
i = 100, loss = 2.4509
starting new epoch
i = 100, loss = 3.0633
starting new epoch
i = 100, loss = 3.0588
starting new epoch
i = 100, loss = 2.9657
starting new epoch
i = 100, loss = 2.8186
starting new epoch
i = 100, loss = 3.0917
starting new epoch
i = 100, loss = 2.0424
starting new epoch
i = 100, loss = 3.6631
starting ne

starting new epoch
i = 100, loss = 1.9763
starting new epoch
i = 100, loss = 2.1237
starting new epoch
i = 100, loss = 2.7786
starting new epoch
i = 100, loss = 2.2155
starting new epoch


In [38]:
print('checking training accuracy...')
checkAccuracy(XY)


checking training accuracy...
Got 4961 / 8000 correct (62.01)


In [39]:
print('checking validation accuracy...')
checkAccuracy(my_dataloaderTe)

checking validation accuracy...
Got 125 / 2662 correct (4.70)


In [40]:
train(50)

/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


i = 100, loss = 2.4784
starting new epoch
i = 100, loss = 1.9646
starting new epoch
i = 100, loss = 1.4863
starting new epoch
i = 100, loss = 2.7162
starting new epoch
i = 100, loss = 2.3571
starting new epoch
i = 100, loss = 2.9301
starting new epoch
i = 100, loss = 2.1042
starting new epoch
i = 100, loss = 1.2779
starting new epoch
i = 100, loss = 1.7552
starting new epoch
i = 100, loss = 0.8972
starting new epoch
i = 100, loss = 1.5916
starting new epoch
i = 100, loss = 1.3580
starting new epoch
i = 100, loss = 1.7536
starting new epoch
i = 100, loss = 1.9626
starting new epoch
i = 100, loss = 1.1654
starting new epoch
i = 100, loss = 2.0342
starting new epoch
i = 100, loss = 2.9617
starting new epoch
i = 100, loss = 1.3506
starting new epoch
i = 100, loss = 2.2686
starting new epoch
i = 100, loss = 2.3137
starting new epoch
i = 100, loss = 1.3706
starting new epoch
i = 100, loss = 1.8966
starting new epoch
i = 100, loss = 1.7058
starting new epoch
i = 100, loss = 1.8096
starting ne

In [41]:
print('checking training accuracy...')
checkAccuracy(XY)
print('checking validation accuracy...')
checkAccuracy(my_dataloaderTe)

checking training accuracy...
Got 4954 / 8000 correct (61.92)
checking validation accuracy...
Got 142 / 2662 correct (5.33)
